# Create a GIF and Animation for Land Surface Temperature Data
ECOSTRESS Tutorials

This code is used to create .gif and .mp4 files to animate ECOSTRESS Land Surface Temperature (LST) images. This code uses Fahrenheit, but it can be modified to create Celsius animations. If you are using macOS, make sure to use 'brew install ffmpeg' in your terminal.

### Import the Necessary Libraries

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rcParams
import rioxarray as rxr
from mpl_toolkits.axes_grid1 import make_axes_locatable
from datetime import datetime
import numpy as np
from matplotlib.animation import PillowWriter
from matplotlib.colors import LinearSegmentedColormap
import pytz

### Define your Input and Output Directories

In [ ]:
# Set ths variable equal to the path where the images you want to create an animation of are stored
image_folder = r'Replace_this_text_with_folder_path'
# Set this variable to the folder or location that you want your output animations to be saved
output_folder = r'Replace_this_text_with_folder_path'

# Create a GIF
### Set Up Functions to Extract UTC Datetime

In [ ]:
# Function to extract title information from filenames
def title_from_file(filename):
    time_info = filename.split('doy')[1][:13]  # Extract date string
    dt = datetime.strptime(time_info, '%Y%j%H%M%S')  # Parse to UTC datetime

    # Convert from UTC to Pacific Time (America/Los_Angeles)
    utc_dt = pytz.utc.localize(dt)
    pacific_tz = pytz.timezone('America/Los_Angeles') # You can change this to the time zone where your images are from
    pacific_dt = utc_dt.astimezone(pacific_tz)

    # Format the date and time to be more readable
    formatted_datetime = pacific_dt.strftime('%Y-%m-%d at %I:%M %p (%Z)')
    title = f"Land Surface Temperature (°F)  {formatted_datetime}"

    return title

# Convert Kelvin to Fahrenheit
def kelvin_to_fahrenheit(kelvin_array):
    return (kelvin_array - 273.15) * 9/5 + 32

# Function to determine if the image is taken during the day or night in Pacific Time
def is_daytime(filename):
    time_info = filename.split('doy')[1][:13]  # Extract date string
    dt = datetime.strptime(time_info, '%Y%j%H%M%S')  # Parse to UTC datetime

    # Convert from UTC to Pacific Time (America/Los_Angeles)
    utc_dt = pytz.utc.localize(dt)
    pacific_tz = pytz.timezone('America/Los_Angeles') # You can change this to the time zone where your images are from
    pacific_dt = utc_dt.astimezone(pacific_tz)

    hour = pacific_dt.hour  # Extract the hour in Pacific Time
    return 6 <= hour <= 18  # Daytime is between 6:00 AM and 6:00 PM Pacific Time

### Identify if Image is Daytime or Nightime

In [ ]:
# Get list of all image files in the folder
files = sorted([f for f in os.listdir(image_folder) if f.endswith('.tif')])

if not files:
    raise FileNotFoundError("No .tif files found in the specified folder.")

# Separate files into daytime and nighttime images
daytime_files = [f for f in files if is_daytime(f)]
nighttime_files = [f for f in files if not is_daytime(f)]

### Make a Helper Function to Create the GIF

In [ ]:
# Define your custom colormap
ET_COLORMAP = LinearSegmentedColormap.from_list("ET", [
    "#f6e8c3",
    "#d8b365",
    "#99974a",
    "#53792d",
    "#6bdfd2",
    "#1839c5"
])

# Helper function to create GIF for a given set of files
def create_gif(file_list, output_file):
    rcParams['animation.html'] = 'jshtml'
    fig = plt.figure()
    ax = fig.add_subplot(111)

    div = make_axes_locatable(ax)
    cax = div.append_axes('right', '5%', '5%')

    # Load LST images, apply scale factor, and convert to Fahrenheit
    LST_rasters = [rxr.open_rasterio(os.path.join(image_folder, file)).squeeze("band", drop=True) * 0.02 for file in file_list]
    LST_rasters_fahrenheit = [kelvin_to_fahrenheit(raster.values) for raster in LST_rasters]

    # Prepare frames for animation
    frames = []
    for raster_fahrenheit in LST_rasters_fahrenheit:
        np.nan_to_num(raster_fahrenheit, copy=False)  # Replace NaN values with 0
        frames.append(raster_fahrenheit)

    # Initialize the first frame for the animation
    cv0 = frames[0]
    im = ax.imshow(cv0, cmap=ET_COLORMAP)
    cb = fig.colorbar(im, cax=cax, label='Land Surface Temperature (°F)')
    tx = ax.set_title(title_from_file(file_list[0]))

    # Use tight_layout() to fix the axis label visibility issue
    plt.tight_layout()

    # Animation function
    def animate(i):
        arr = frames[i]
        vmax = np.max(arr[arr > 0.0])
        vmin = np.min(arr[arr > 0.0])
        im.set_data(arr)
        im.set_clim(vmin, vmax)
        tx.set_text(title_from_file(file_list[i]))
        return im, tx

    # Create the animation
    ani = animation.FuncAnimation(fig, animate, frames=len(file_list), interval=1000, blit=True, repeat_delay=5000)

    # Save the animation as a GIF
    print(f"Saving animation to {output_file}")
    ani.save(output_file, writer=PillowWriter(fps=1))
    print(f"Animation successfully saved to {output_file}")


### Save the Outputs

In [ ]:
# Create GIF for daytime images
if daytime_files:
    output_path = os.path.join(output_folder, 'LST_animation_daytime.gif')
    create_gif(daytime_files, output_path)

# Create GIF for nighttime images
if nighttime_files:
    output_path = os.path.join(output_folder, 'LST_animation_nighttime.gif')
    create_gif(nighttime_files, output_path)

# Create Animations

### Set Up Functions to Extract UTC Datetime

In [ ]:
# Function to extract title information from filenames
def title_from_file(filename):
    time_info = filename.split('doy')[1][:13]  # '2024002071024'
    dt = datetime.strptime(time_info, '%Y%j%H%M%S')  # Parse to UTC datetime

    # Convert from UTC to Pacific Time (America/Los_Angeles)
    utc_dt = pytz.utc.localize(dt)
    pacific_tz = pytz.timezone('America/Los_Angeles') # You can change this to the time zone where your images are from
    pacific_dt = utc_dt.astimezone(pacific_tz)

    # Format the date and time to be more readable
    formatted_datetime = pacific_dt.strftime('%Y-%m-%d at %I:%M %p (%Z)')
    title = f"Land Surface Temperature (°F)  {formatted_datetime}"

    return title

# Convert Kelvin to Fahrenheit
def kelvin_to_fahrenheit(kelvin_array):
    return (kelvin_array - 273.15) * 9/5 + 32

# Function to determine if the image is taken during the day or night in Pacific Time
def is_daytime(filename):
    time_info = filename.split('doy')[1][:13]  # Extract date string
    dt = datetime.strptime(time_info, '%Y%j%H%M%S')  # Parse to UTC datetime

    # Convert from UTC to Pacific Time (America/Los_Angeles)
    utc_dt = pytz.utc.localize(dt)
    pacific_tz = pytz.timezone('America/Los_Angeles') # You can change this to the time zone where your images are from
    pacific_dt = utc_dt.astimezone(pacific_tz)

    hour = pacific_dt.hour  # Extract the hour in Pacific Time
    return 6 <= hour <= 18  # Daytime is between 6:00 AM and 6:00 PM Pacific Time

### Identify if Image is Daytime or Nighttime

In [ ]:
# Get list of all image files in the folder
files = sorted([f for f in os.listdir(image_folder) if f.endswith('.tif')])

# Separate files into daytime and nighttime images
daytime_files = [f for f in files if is_daytime(f)]
nighttime_files = [f for f in files if not is_daytime(f)]

### Helper Function to Create .mp4 File

In [ ]:
# Define your custom colormap
ET_COLORMAP = LinearSegmentedColormap.from_list("ET", [
    "#f6e8c3",
    "#d8b365",
    "#99974a",
    "#53792d",
    "#6bdfd2",
    "#1839c5"
])

# Helper function to create the animation
def create_animation(file_list, output_file):
    rcParams['animation.html'] = 'jshtml'
    fig = plt.figure()
    ax = fig.add_subplot(111)

    div = make_axes_locatable(ax)
    cax = div.append_axes('right', '5%', '5%')

    # Load LST images, apply scale factor, and convert to Fahrenheit
    LST_rasters = [(rxr.open_rasterio(os.path.join(image_folder, file)).squeeze("band", drop=True) * 0.02) for file in file_list]
    LST_rasters_fahrenheit = [kelvin_to_fahrenheit(raster.values) for raster in LST_rasters]

    # Prepare frames for animation
    frames = []
    for raster_fahrenheit in LST_rasters_fahrenheit:
        np.nan_to_num(raster_fahrenheit, copy=False)  # Replace NaN values with 0
        frames.append(raster_fahrenheit)

    # Initialize the first frame for the animation
    cv0 = frames[0]
    im = ax.imshow(cv0, cmap=ET_COLORMAP)
    cb = fig.colorbar(im, cax=cax, label='Land Surface Temperature (°F)')
    tx = ax.set_title(title_from_file(file_list[0]))

    # Use tight_layout() to fix the axis label visibility issue
    plt.tight_layout()

    # Animation function
    def animate(i):
        arr = frames[i]
        vmax = np.max(arr[arr > 0.0])
        vmin = np.min(arr[arr > 0.0])
        im.set_data(arr)
        im.set_clim(vmin, vmax)
        tx.set_text(title_from_file(file_list[i]))
        return im, tx

    # Create the animation
    ani = animation.FuncAnimation(fig, animate, frames=len(file_list), interval=1000, blit=True, repeat_delay=5000)

    # Save the animation as an MP4
    print(f"Saving animation to {output_file}")
    ani.save(output_file, writer='ffmpeg', fps=2)
    print(f"Animation successfully saved to {output_file}")


### Save the Outputs

In [ ]:
# Create GIF for daytime images
if daytime_files:
    output_path = os.path.join(output_folder, 'LST_animation_daytime.mp4')
    create_animation(daytime_files, output_path)

# Create GIF for nighttime images
if nighttime_files:
    output_path = os.path.join(output_folder, 'LST_animation_nighttime.mp4')
    create_animation(nighttime_files, output_path)

# Create Animations with Standardized Values
If you need to create the animations but with standardized values, the following code cells provide the steps to do so. The process is very similar to what we just ran, however you will be creating and adjusting the min-max values for the ET data that you are working with.

### Set up Functions to Extract UTC Datetime

In [ ]:
# Function to extract title information from filenames
def title_from_file(filename):
    time_info = filename.split('doy')[1][:13]  # '2024002071024'
    dt = datetime.strptime(time_info, '%Y%j%H%M%S')  # Parse to UTC datetime

    # Convert from UTC to Pacific Time (America/Los_Angeles)
    utc_dt = pytz.utc.localize(dt)
    pacific_tz = pytz.timezone('America/Los_Angeles') # You can change this to the time zone where your images are from
    pacific_dt = utc_dt.astimezone(pacific_tz)

    # Format the date and time to be more readable
    formatted_datetime = pacific_dt.strftime('%Y-%m-%d at %I:%M %p (%Z)')
    title = f"Land Surface Temperature (°F)  {formatted_datetime}"

    return title

# Convert Kelvin to Fahrenheit
def kelvin_to_fahrenheit(kelvin_array):
    return (kelvin_array - 273.15) * 9/5 + 32

# Function to determine if the image is taken during the day or night in Pacific Time
def is_daytime(filename):
    time_info = filename.split('doy')[1][:13]  # Extract date string
    dt = datetime.strptime(time_info, '%Y%j%H%M%S')  # Parse to UTC datetime

    # Convert from UTC to Pacific Time (America/Los_Angeles)
    utc_dt = pytz.utc.localize(dt)
    pacific_tz = pytz.timezone('America/Los_Angeles') # You can change this to the time zone where your images are from
    pacific_dt = utc_dt.astimezone(pacific_tz)

    hour = pacific_dt.hour  # Extract the hour in Pacific Time
    return 6 <= hour <= 18  # Daytime is between 6:00 AM and 6:00 PM Pacific Time

### Identify if Image is Daytime or Nighttime

In [ ]:
# Get list of all image files in the folder
files = sorted([f for f in os.listdir(image_folder) if f.endswith('.tif')])

# Separate files into daytime and nighttime images
daytime_files = [f for f in files if is_daytime(f)]
nighttime_files = [f for f in files if not is_daytime(f)]

### Helper Function to Create Standardized .mp4 File

In [ ]:
# Define your custom colormap
ET_COLORMAP = LinearSegmentedColormap.from_list("ET", [
    "#f6e8c3",
    "#d8b365",
    "#99974a",
    "#53792d",
    "#6bdfd2",
    "#1839c5"
])

# Helper function to create the animation with fixed vmin and vmax
def create_animation(file_list, output_file, vmin, vmax):
    rcParams['animation.html'] = 'jshtml'
    fig = plt.figure()
    ax = fig.add_subplot(111)

    div = make_axes_locatable(ax)
    cax = div.append_axes('right', '5%', '5%')

    # Load LST images, apply scale factor, and convert to Fahrenheit
    LST_rasters = [(rxr.open_rasterio(os.path.join(image_folder, file)).squeeze("band", drop=True) * 0.02) for file in file_list]
    LST_rasters_fahrenheit = [kelvin_to_fahrenheit(raster.values) for raster in LST_rasters]

    # Prepare frames for animation
    frames = []
    for raster_fahrenheit in LST_rasters_fahrenheit:
        np.nan_to_num(raster_fahrenheit, copy=False)  # Replace NaN values with 0
        frames.append(raster_fahrenheit)

    # Initialize the first frame for the animation
    cv0 = frames[0]
    im = ax.imshow(cv0, cmap=ET_COLORMAP, vmin=vmin, vmax=vmax)  # Set vmin and vmax for fixed scale
    cb = fig.colorbar(im, cax=cax, label='Land Surface Temperature (°F)')
    tx = ax.set_title(title_from_file(file_list[0]))

    # Use tight_layout() to fix the axis label visibility issue
    plt.tight_layout()

    # Animation function
    def animate(i):
        arr = frames[i]
        im.set_data(arr)
        tx.set_text(title_from_file(file_list[i]))
        return im, tx

    # Create the animation
    ani = animation.FuncAnimation(fig, animate, frames=len(file_list), interval=1000, blit=True, repeat_delay=5000)

    # Save the animation as an MP4
    print(f"Saving animation to {output_file}")
    ani.save(output_file, writer='ffmpeg', fps=2)
    print(f"Animation successfully saved to {output_file}")


### Save the Standardized Outputs

In [ ]:
# Create daytime animation with fixed vmin=45 and vmax=130
if daytime_files:
    output_path = os.path.join(output_folder, 'LST_animation_daytime_fixed_scale.mp4')
    create_animation(daytime_files, output_path, vmin=45, vmax=130)

# Create nighttime animation with fixed vmin=30 and vmax=90
if nighttime_files:
    output_path = os.path.join(output_folder, 'LST_animation_nighttime_fixed_scale.mp4')
    create_animation(nighttime_files, output_path, vmin=45, vmax=130)